Получаем данные и импортим необходимые библиотеки

In [121]:
import numpy as np
import pandas as pd

wine_train = pd.read_csv('winequality-red.csv', sep=";")
wine_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Разбиваем на матрицу объектов-признаков и вектор правильных ответов

In [122]:
X = wine_train.drop(['quality'], axis='columns').values
y = wine_train['quality'].values

Применяем отложенную выборку. Т.е. разделяем предоставленные данные на обучающие и валидационные(80/20)

In [123]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=True, random_state=9, test_size=0.2, stratify=y)

Подготоваливаем матрицу объектов-признаков

In [124]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)

Построим модель используя алгоритм Метод ближайших соседей.

In [125]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knnc = KNeighborsClassifier(algorithm='auto')
knnc = knnc.fit(X_train, y_train)

predictions = knnc.predict(X_val)
acc = accuracy_score(y_val, predictions)
print(acc * 100.0)

59.375


Настроим число соседей на 5кратной кросс-валидации. 

In [126]:
from sklearn.model_selection import GridSearchCV, cross_val_score

knn_params = {'n_neighbors': range(1,20)}
knn_grid = GridSearchCV(knnc, knn_params, cv=5, n_jobs=-1, verbose=True)

knn_grid.fit(X_train, y_train)
knn_grid.best_params_

Fitting 5 folds for each of 19 candidates, totalling 95 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done  95 out of  95 | elapsed:   21.6s finished


{'n_neighbors': 1}

In [127]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knnc = KNeighborsClassifier(n_neighbors=1, algorithm='auto')
knnc = knnc.fit(X_train, y_train)

predictions = knnc.predict(X_val)
acc = accuracy_score(y_val, predictions)
print(acc * 100.0)

66.875


Построим модель используя алгоритм Дерева решений. Также применим адаптивный бустинг для Дерева решений.

In [136]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dt1 = DecisionTreeClassifier(criterion='entropy', random_state=12345)
dt1 = dt1.fit(X_train, y_train)

predictions1 = dt1.predict(X_val)
acc1 = accuracy_score(y_val, predictions1)
print(acc1 *100.0)

dt2 = DecisionTreeClassifier(criterion='gini', random_state=1337)
dt2 = dt2.fit(X_train, y_train)

predictions2 = dt2.predict(X_val)
acc2 = accuracy_score(y_val, predictions2)
print(acc2 *100.0)

abdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=11, random_state=228),
                          algorithm="SAMME",
                          random_state=228)

abdt.fit(X_train, y_train)
pred = abdt.predict(X_val)
acc0 = accuracy_score(y_val, pred)
print(acc0 * 100.0)

62.5
65.0
71.875


Настроим максимальную глубину дерева на 5кратной кросс-валидации.

In [137]:
from sklearn.model_selection import GridSearchCV, cross_val_score

tree_params1 = {'max_depth': range(1,50)}
tree_grid1 = GridSearchCV(dt1, tree_params1, cv=5, verbose=True)
tree_grid1.fit(X_train, y_train)
tree_grid1.best_params_

Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:    8.5s finished


{'max_depth': 17}

In [138]:
tree_params2 = {'max_depth': range(1,50)}
tree_grid2 = GridSearchCV(dt2, tree_params2, cv=5, verbose=True)
tree_grid2.fit(X_train, y_train)
tree_grid2.best_params_

Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:    4.4s finished


{'max_depth': 16}

In [139]:
dt1 = DecisionTreeClassifier(criterion='entropy', max_depth=17, random_state=12345)
dt1 = dt1.fit(X_train, y_train)
predictions1 = dt1.predict(X_val)
acc1 = accuracy_score(y_val, predictions1)
print(acc1 *100.0)

dt2 = DecisionTreeClassifier(criterion='gini', max_depth=16, random_state=1337)
dt2 = dt2.fit(X_train, y_train)
predictions2 = dt2.predict(X_val)
acc2 = accuracy_score(y_val, predictions2)
print(acc2 *100.0)

62.5
68.125


Построим модель используя алгоритм Случайный лес.

In [140]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=16, random_state=100)
rf = rf.fit(X_train, y_train)

predictions4 = rf.predict(X_val)
acc4 = accuracy_score(y_val, predictions4)
print(acc4 * 100)

69.6875


Настроим число деревьев на 5кратной кросс-валидации.

In [142]:
tree_params4 = {'n_estimators': range(1,100)}
tree_grid4 = GridSearchCV(rf, tree_params4, cv=5, verbose=True)
tree_grid4.fit(X_train, y_train)
tree_grid4.best_params_

Fitting 5 folds for each of 99 candidates, totalling 495 fits


[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:  3.6min finished


{'n_estimators': 88}

In [143]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=88, random_state=100)
rf = rf.fit(X_train, y_train)

predictions4 = rf.predict(X_val)
acc4 = accuracy_score(y_val, predictions4)
print(acc4 * 100)

73.125


Построим модель используя алгоритм Градиентный бустинг.

In [161]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=64, random_state=228)
gbm.fit(X_train, y_train)

pred = gbm.predict(X_val)
acc5 = accuracy_score(y_val, pred)
print(acc5 * 100.0)

68.4375
